<a href="https://colab.research.google.com/github/adupatil/Physical-trait-recognition-of-fictional-characters/blob/main/NER_Novel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers

In [2]:
! pip install datasets

In [31]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import datasets
import pandas as pd
import numpy as np

In [4]:
dataset = datasets.load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
train = dataset['train']

In [6]:
ner_labels = train.features["ner_tags"].feature.names

In [7]:
ner_labels

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [8]:
words = train[4]["tokens"]
labels = train[4]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = ner_labels[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer . 
B-LOC   O  O              O  O   B-ORG    I-ORG O  O          O         B-PER  I-PER     O    O  O         O         O      O   O         O    O         O     O    B-LOC   O     O   O          O      O   O       O 


In [9]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [10]:
inputs = tokenizer(train[0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [11]:
## Word ids of each token
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [12]:
## Gives new tags to the updated tokens after using AutoTokenizer
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [13]:
ner_tags = train[0]["ner_tags"]
word_ids = inputs.word_ids()
align_labels_with_tokens(ner_tags,word_ids)

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]

In [14]:
## aligning all labels with tokens for our training data
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs 

In [15]:
## Map the tokenize_and_align_labels function to dataset with  batches for faster tokenization of the dataset
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train.column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [16]:
tokenized_datasets['train'][1]

{'input_ids': [101, 1943, 14428, 102],
 'token_type_ids': [0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1],
 'labels': [-100, 1, 2, -100]}

In [17]:
## Data collation
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, return_tensors="tf"
)

In [18]:
## pads the remaining values with -100 so that those predictions are not used while computing loss and also keeps all the inputs of same size
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


<tf.Tensor: shape=(2, 12), dtype=int64, numpy=
array([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0,
        -100],
       [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100,
        -100]])>

In [19]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

In [ ]:
!pip install seqeval
!pip install evaluate

In [22]:
# id2label and label2id, which contain the mapping from ID to label and vice versa
id2label = {i: label for i, label in enumerate(ner_labels)}
label2id = {v: k for k, v in id2label.items()}

In [23]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
)

All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
model.config.num_labels

9

In [25]:
from transformers import create_optimizer
import tensorflow as tf

# Train in mixed-precision float16
# Comment this line out if you're not using a GPU 
tf.keras.mixed_precision.set_global_policy("mixed_float16")

'''The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.'''
num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [32]:
## fitiing the model
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(output_dir="bert-finetuned-ner-adwait", tokenizer=tokenizer)

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=num_epochs,
)

Cloning https://huggingface.co/Adwait1199/bert-finetuned-ner-adwait into local empty directory.


Epoch 1/3


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:717: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


878/878 [==============================] - 206s 235ms/step - loss: 0.0220 - val_loss: 0.0522
Epoch 2/3


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:717: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


878/878 [==============================] - 225s 256ms/step - loss: 0.0222 - val_loss: 0.0522
Epoch 3/3


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:717: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


878/878 [==============================] - 233s 266ms/step - loss: 0.0225 - val_loss: 0.0522


In [27]:
## Metrics for evaluating predictions
import evaluate

metric = evaluate.load("seqeval")

In [28]:
all_predictions = []
all_labels = []
for batch in tf_eval_dataset:
    logits = model.predict_on_batch(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            all_predictions.append(ner_labels[predicted_idx])
            all_labels.append(ner_labels[label_idx])
metric.compute(predictions=[all_predictions], references=[all_labels])

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:717: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


{'LOC': {'precision': 0.9534632034632035,
  'recall': 0.9591725639629831,
  'f1': 0.9563093622795116,
  'number': 1837},
 'MISC': {'precision': 0.8542094455852156,
  'recall': 0.9023861171366594,
  'f1': 0.8776371308016877,
  'number': 922},
 'ORG': {'precision': 0.8873744619799139,
  'recall': 0.9224459358687547,
  'f1': 0.9045703839122486,
  'number': 1341},
 'PER': {'precision': 0.9576407506702412,
  'recall': 0.9695982627578719,
  'f1': 0.9635824116536282,
  'number': 1842},
 'overall_precision': 0.9236967604012498,
 'overall_recall': 0.945304611242006,
 'overall_f1': 0.9343757797554687,
 'overall_accuracy': 0.9857390946017542}

In [34]:
## Predict on single text
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "Adwait1199/bert-finetuned-ner-adwait"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

Some layers from the model checkpoint at Adwait1199/bert-finetuned-ner-adwait were not used when initializing TFBertForTokenClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at Adwait1199/bert-finetuned-ner-adwait.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


[{'entity_group': 'PER',
  'score': 0.99299747,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.95198506,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.99746406,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [37]:
with open("/content/alice.txt",'r') as file:
  lines = file.readlines()

In [38]:
lines[0]

['Alice’s Adventures in Wonderland is a classic novel written by Lewis Carroll. It tells the story of a young girl named Alice who falls through a rabbit hole into a fantasy world filled with peculiar creatures and nonsensical situations. The following is a 1000-word excerpt from the book.Alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, "and what is the use of a book," thought Alice, "without pictures or conversations?"So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid) whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.There was nothing so very remarkable in that, nor did Alice think it so very much out of the way to hear the Rabbit say 

In [39]:
token_classifier(lines[0])

[{'entity_group': 'MISC',
  'score': 0.8699649,
  'word': 'Alice ’ s',
  'start': 0,
  'end': 7},
 {'entity_group': 'MISC',
  'score': 0.785856,
  'word': 'Adventures in Wonderland',
  'start': 8,
  'end': 32},
 {'entity_group': 'PER',
  'score': 0.85331655,
  'word': 'Lewis Carroll',
  'start': 63,
  'end': 76},
 {'entity_group': 'PER',
  'score': 0.97048426,
  'word': 'Alice',
  'start': 119,
  'end': 124},
 {'entity_group': 'PER',
  'score': 0.891689,
  'word': 'Alice',
  'start': 288,
  'end': 293},
 {'entity_group': 'PER',
  'score': 0.74248177,
  'word': 'Alice',
  'start': 547,
  'end': 552},
 {'entity_group': 'MISC',
  'score': 0.89707416,
  'word': 'White Rabbit',
  'start': 833,
  'end': 845},
 {'entity_group': 'PER',
  'score': 0.48284772,
  'word': 'Alice',
  'start': 932,
  'end': 937},
 {'entity_group': 'PER',
  'score': 0.5271118,
  'word': 'Alice',
  'start': 1488,
  'end': 1493},
 {'entity_group': 'PER',
  'score': 0.47794595,
  'word': 'Alice',
  'start': 1678,
  'end